In [ ]:
import cv2
import numpy as np
import os

# Load the image
image_path = './opal2.jpeg'
assert os.path.exists(image_path)

img = cv2.imread(image_path)

In [ ]:
import cv2
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Hide axis
plt.show()

In [ ]:
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.convertScaleAbs(gray)
median_blur = cv2.medianBlur(gray, 5)
plt.imshow(cv2.cvtColor(median_blur, cv2.COLOR_BGR2RGB))

edges = cv2.Canny(median_blur, 100, 200)
plt.imshow(cv2.cvtColor(edges, cv2.COLOR_BGR2RGB))

In [ ]:
ret, thresh = cv2.threshold(edges, 127, 255, 0)
plt.imshow(cv2.cvtColor(thresh, cv2.COLOR_BGR2RGB))
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Find the biggest contour
print(len(contours))

cv2.drawContours(img, contours, -1, (0,255,0), 3)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
image = img
vertical_line_length_px = None

# Iterate through contours
for contour in contours:
    # Find the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)
    
    # Compute the center coordinates
    cx = x + w // 2
    cy = y + h // 2
    
    # Compute the total height and width
    total_height = h
    total_width = w
    
    # Draw horizontal and vertical lines through the center
    cv2.line(image, (x, cy), (x + w, cy), (255, 255, 0), 2)  # Horizontal line
    cv2.line(image, (cx, y), (cx, y + h), (255, 255, 0), 2)  # Vertical line
    vertical_line_length_px= h
    
    break
    
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))



In [ ]:
# Ask user for the length of the vertical line in mm
vertical_line_length_mm = float(input("Enter the length of the vertical line in mm: "))

In [ ]:


# Calculate conversion factor for pixels to mm

px_to_mm_vertical = vertical_line_length_mm / vertical_line_length_px

# Draw grid
grid_size_mm = 1
grid_color = (0, 0, 255)  # Color of the grid lines
for i in range(0, image.shape[0], int(grid_size_mm / px_to_mm_vertical)):
    cv2.line(image, (0, i), (image.shape[1], i), grid_color, 1)  # Horizontal lines
for j in range(0, image.shape[1], int(grid_size_mm / px_to_mm_vertical)):
    cv2.line(image, (j, 0), (j, image.shape[0]), grid_color, 1)  # Vertical lines
    
# Display total height and width
cv2.putText(image, f'{total_height * px_to_mm_vertical} mm', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
cv2.putText(image, f'{total_width * px_to_mm_vertical}mm', (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
print(f'Total Height: {total_height * px_to_mm_vertical} mm')
print(f'Total Width: {total_width * px_to_mm_vertical}mm')

# Define offset (in pixels)
offset = 50

# Iterate through contours
for contour in contours:
    # Get bounding box coordinates
    x, y, w, h = cv2.boundingRect(contour)
    
    # Add offset to bounding box coordinates
    x -= offset
    y -= offset
    w += 2 * offset
    h += 2 * offset
    
    # Ensure the bounding box coordinates are within the image bounds
    x = max(0, x)
    y = max(0, y)
    w = min(w, image.shape[1] - x)
    h = min(h, image.shape[0] - y)
    
    # Crop the region around the contour with offset
    cropped_image = image[y:y+h, x:x+w]
    
    plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
    break
